![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Stage 1: Install dependencies and setting up GPU environment

In [ ]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 33kB/s 
     |████████████████████████████████| 419kB 41.1MB/s 
     |████████████████████████████████| 3.0MB 46.7MB/s 
     |████████████████████████████████| 61kB 28.4MB/s 


In [ ]:
!pip install tqdm

### Downloading the Dogs vs Cats dataset 

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-10-24 06:57:43--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 74.125.142.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   136MB/s    in 0.5s    

2020-10-24 06:57:43 (136 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Stage 2: Dataset preprocessing

### Import project dependencies

In [2]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.3.0'

### Unzipping the Dogs vs Cats dataset

In [3]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [4]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [5]:
zip_object.extractall("./")

In [6]:
zip_object.close()

### Seting up dataset paths

In [7]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [8]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Building the model

### Loading the pre-trained model (MobileNetV2)

In [9]:
IMG_SHAPE = (128, 128, 3)

In [10]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [11]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

### Freezing the base model

In [12]:
base_model.trainable = False

### Defining the custom head for our network

In [13]:
base_model.output

<tf.Tensor 'out_relu/Relu6:0' shape=(None, 4, 4, 1280) dtype=float32>

In [14]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [16]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(None, 1280) dtype=float32>

In [17]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Defining the model

In [18]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [19]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________________

### Compiling the model

In [20]:
# we are using a pretrained network so we have to use samll learning rate lr
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Creating Data Generators

Resizing images

    Big pre-trained architecture support only certain input sizes.

For example: MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [21]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [22]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [23]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Training the model

In [30]:
model.fit(train_generator, epochs=25, validation_data=valid_generator)

Epoch 1/10
16/16 [==============================] - 8s 530ms/step - loss: 0.1487 - accuracy: 0.9520 - val_loss: 0.1465 - val_accuracy: 0.9460
Epoch 2/10
16/16 [==============================] - 8s 507ms/step - loss: 0.1435 - accuracy: 0.9530 - val_loss: 0.1416 - val_accuracy: 0.9460
Epoch 3/10
16/16 [==============================] - 8s 507ms/step - loss: 0.1385 - accuracy: 0.9535 - val_loss: 0.1371 - val_accuracy: 0.9490
Epoch 4/10
16/16 [==============================] - 8s 506ms/step - loss: 0.1344 - accuracy: 0.9560 - val_loss: 0.1332 - val_accuracy: 0.9490
Epoch 5/10
16/16 [==============================] - 8s 507ms/step - loss: 0.1304 - accuracy: 0.9560 - val_loss: 0.1295 - val_accuracy: 0.9500
Epoch 6/10
16/16 [==============================] - 8s 504ms/step - loss: 0.1270 - accuracy: 0.9575 - val_loss: 0.1263 - val_accuracy: 0.9510
Epoch 7/10
16/16 [==============================] - 8s 504ms/step - loss: 0.1237 - accuracy: 0.9580 - val_loss: 0.1235 - val_accuracy: 0.9510
Epoch 

### Transfer learning model evaluation

In [31]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)

8/8 [==============================] - 2s 311ms/step - loss: 0.1163 - accuracy: 0.9540


In [32]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9539999961853027


## Fine tuning


There are a few pointers:

- DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. The goal of the Fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
- Start with the fine tunning AFTER you have finished with transfer learning step. If we try to perform Fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model. 

### Un-freeze a few top layers from the model

In [33]:
base_model.trainable = True

In [34]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 155


In [36]:
# Starting layer for fine tuning
fine_tune_at = 100

In [37]:
#  we are setting all layers from 0-100 to be not trained(fine tuned)
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compiling the model for fine-tuning

In [38]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Fine tuning

In [39]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 9s 564ms/step - loss: 0.1588 - accuracy: 0.9310 - val_loss: 0.0668 - val_accuracy: 0.9720
Epoch 2/5
16/16 [==============================] - 8s 522ms/step - loss: 0.0231 - accuracy: 0.9970 - val_loss: 0.0685 - val_accuracy: 0.9720
Epoch 3/5
16/16 [==============================] - 8s 510ms/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 0.0764 - val_accuracy: 0.9720
Epoch 4/5
16/16 [==============================] - 8s 518ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.0810 - val_accuracy: 0.9720
Epoch 5/5
16/16 [==============================] - 8s 512ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.1003 - val_accuracy: 0.9720


### Evaluating the fine tuned model

In [40]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [41]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.972000002861023
